![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/17.Graph_builder_for_DL_models.ipynb)

**NOTE**: 

All the components in this notebook work properly in these versions:

*   sparknlp: 3.4.2
*   sparknlp_jsl: 3.4.2
*   pyspark: 3.1.2

Please keep in mind you may have some issues if you use any other version.

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import sparknlp
import sparknlp_jsl

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.4.2


In [ ]:
from sparknlp_jsl.training import tf_graph


# before sparknlp_jsl 3.2.1 version run the code below
# %tensorflow_version 1.x

In [ ]:
tf_graph.get_models()


['ner_dl', 'generic_classifier', 'assertion_dl', 'relation_extraction']

In [ ]:
!pip install -q tensorflow==2.7.0
!pip install -q tensorflow_addons

## NER_DL

In [ ]:
tf_graph.print_model_params("ner_dl")

ner_dl parameters.
Parameter            Required   Default value        Description
ntags                yes        -                    Number of tags.
embeddings_dim       no         200                  Embeddings dimension.
nchars               no         100                  Number of chars.
lstm_size            no         128                  Number of LSTM units.
gpu_device           no         0                    Device for training.
is_medical           no         0                    Build a Medical Ner graph.


In [ ]:
tf_graph.build("ner_dl",
               build_params={"embeddings_dim": 200, 
                             "nchars": 80, 
                             "ntags": 12, 
                             "is_medical": 1}, 
               model_location="./medical_ner_graphs",
               model_filename="auto")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1702: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


ner_dl graph exported to ./medical_ner_graphs/blstm_12_200_128_80.pb


## AssertionDL

In [ ]:
tf_graph.print_model_params("assertion_dl")


assertion_dl parameters.
Parameter            Required   Default value        Description
max_seq_len          no         250                  Maximum sequence length.
feat_size            no         200                  Feature size.
n_classes            yes        -                    Number of classes.
device               no         /cpu:0               Device for training.
n_hidden             no         34                   Number of hidden units.


In [ ]:
tf_graph.build("assertion_dl",
               build_params={"n_classes": 10}, 
               model_location="./assertion_graph",
               model_filename="auto")

Device mapping: no known devices.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:715: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1702: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Device mapping: no known devices.
assertion_dl graph exported to ./assertion_graph/blstm_34_32_30_200_10.pb


## GenericClassifier

In [ ]:
tf_graph.print_model_params("generic_classifier")

generic_classifier parameters.
Parameter            Required   Default value        Description
hidden_layers        no         [200]                List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim            yes        -                    Input dimension.
output_dim           yes        -                    Output dimension.
hidden_act           no         relu                 Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2        no         0                    L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2    no         0                    L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm           no         0                    Batch normalization. Boolean (0 or 1).


In [ ]:
tf_graph.build("generic_classifier",
               build_params={"input_dim": 100, 
                             "output_dim": 10, 
                             "hidden_layers": [300, 200, 100], 
                             "hidden_act": "tanh"}, 
               model_location="generic_graph", 
               model_filename="auto")

generic_classifier graph exported to generic_graph/gcl.100.10.pb


## RelationExtraction

In [ ]:
tf_graph.print_model_params("relation_extraction")

relation_extraction parameters.
Parameter            Required   Default value        Description
hidden_layers        no         [200]                List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim            yes        -                    Input dimension.
output_dim           yes        -                    Output dimension.
hidden_act           no         relu                 Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2        no         0                    L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2    no         0                    L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm           no         0                    Batch normalization. Boolean (0 or 1).


In [ ]:
tf_graph.build("relation_extraction",
               build_params={"input_dim": 6000, 
                             "output_dim": 3, 
                             'batch_norm':1, 
                             "hidden_layers": [300, 200], 
                             "hidden_act": "relu", 
                             'hidden_act_l2':1}, 
               model_location="relation_graph", 
               model_filename="re_with_BN.pb")

Instructions for updating:
Colocations handled automatically by placer.
relation_extraction graph exported to relation_graph/re_with_BN.pb
